In [7]:
# ============================
# 0. Cài thư viện cần thiết
# ============================
!pip install joblib scikit-learn

# ============================
# 1. Import
# ============================
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib

# ============================
# 2. Sinh dữ liệu giống MATLAB
# ============================
nSample = 500
rho, c, Hc = 2400, 0.84, 250
E, ft, alpha_T = 40000, 4.0, 1e-5

B = 2 + (30-2)*np.random.rand(nSample)
L = 2 + (30-2)*np.random.rand(nSample)
H = 2 + (7-2)*np.random.rand(nSample)

H_top, H_bot = H/5, 4*H/5
V_top = B*L*H_top
V_bot = B*L*H_bot
V_total = V_top + V_bot

C_top = 370 + (450-370)*np.random.rand(nSample)
C_bot = 300 + (360-300)*np.random.rand(nSample)

T_env  = 15 + (35-15)*np.random.rand(nSample)
T_init = 15 + (30-15)*np.random.rand(nSample)
T_form = 20 + (50-20)*np.random.rand(nSample)

# Tmax
Q_top = Hc * C_top * V_top
Q_bot = Hc * C_bot * V_bot
Q_total = Q_top + Q_bot
Tmax_center = T_init + Q_total / (rho * c * V_total)

# ΔT
S = 2*(B*H + B*L + H*L)
VS = V_total / S
C_avg = (C_top + C_bot)/2
alpha, gamma, beta = 25, 0.1, 1.2
DeltaT_surface = alpha * VS * (C_avg/350) + gamma*(T_env - T_form)
DeltaT_corner = beta * DeltaT_surface

# CI
sigma_th = E * alpha_T * DeltaT_surface
CI = ft / sigma_th

# Noise
noise_level = 0.0005
Tmax_center    *= (1 + noise_level*np.random.randn(nSample))
DeltaT_surface *= (1 + noise_level*np.random.randn(nSample))
DeltaT_corner  *= (1 + noise_level*np.random.randn(nSample))
CI             *= (1 + noise_level*np.random.randn(nSample))

# Dữ liệu train
X = np.vstack([B, H, L, C_top, C_bot, T_env, T_init, T_form]).T
Y = np.vstack([Tmax_center, DeltaT_surface, DeltaT_corner, CI]).T

# ============================
# 3. Chuẩn hóa dữ liệu
# ============================
scalerX = StandardScaler()
scalerY = StandardScaler()
X_scaled = scalerX.fit_transform(X)
Y_scaled = scalerY.fit_transform(Y)

# ============================
# 4. Train ANN
# ============================
model = MLPRegressor(
    hidden_layer_sizes=(12,8),
    solver='lbfgs',       # tốt cho dataset nhỏ
    max_iter=20000,
    tol=1e-6,
    random_state=42
)
model.fit(X_scaled, Y_scaled)

# ============================
# 5. Đánh giá sai số
# ============================
Y_pred = model.predict(X_scaled)
MSE = mean_squared_error(Y_scaled, Y_pred)
print(f"Sai số huấn luyện MSE = {MSE:.6f}")

# ============================
# 6. Lưu mô hình + scaler
# ============================
joblib.dump({"model": model, "scalerX": scalerX, "scalerY": scalerY}, "ANN_model.pkl")
print(">>> Đã lưu ANN_model.pkl (gồm model + scalerX + scalerY)")



Sai số huấn luyện MSE = 0.003383
>>> Đã lưu ANN_model.pkl (gồm model + scalerX + scalerY)
